In [188]:
import time  
import autoit

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import pandas as pd

In [156]:
# launch challenge page

path_dirver = r'F:\ML\HCL\HCL ML Challenge\chromedriver.exe'

browser = webdriver.Chrome(path_dirver)
browser.get('https://www.hackerearth.com/challenges/hiring/hcl-ml-hiring-2020/') 

time.sleep(2)

In [157]:
# close all other open tabs

original_window = browser.current_window_handle

for window_handle in browser.window_handles:
    if window_handle != original_window:
        browser.switch_to.window(window_handle)
        browser.close()
        break

browser.switch_to.window(original_window)

In [158]:
# login with google

email = 'example@gmail.com'
pwd = 'pwd'

login_modal = browser.find_element_by_xpath('/html/body/header/nav/ul[2]/li[2]/a')
login_modal.click()
time.sleep(1)
assert len(browser.window_handles) == 1

wait = WebDriverWait(browser, 10)

browser.find_element_by_xpath('//*[@id="login-modal"]/div[2]/div[1]/div[1]/a').click()
wait.until(EC.number_of_windows_to_be(2))

for window_handle in browser.window_handles:
    if window_handle != original_window:
        browser.switch_to.window(window_handle)
        break

tbox_email = browser.find_element_by_xpath('//*[@id="identifierId"]')
tbox_email.send_keys(eamil)

browser.find_element_by_xpath('//*[@id="identifierNext"]/span/span').click()

time.sleep(2)

tbox_pass = browser.find_element_by_xpath('//*[@id="password"]/div[1]/div/div[1]/input')
tbox_pass.send_keys(pwd)

browser.find_element_by_xpath('//*[@id="passwordNext"]/span/span').click()
wait.until(EC.number_of_windows_to_be(1))


browser.switch_to.window(original_window)

In [159]:
# launch challenge

browser.find_element_by_xpath('/html/body/div[11]/div/div[1]/div[2]/div[2]/div/a').click()

In [161]:
# load complete submission file

# set file path for complete submission file
filepath = 'test_sub.csv'

results = pd.read_csv(filepath)

scores = results.copy()

In [172]:
# add column to store score of each file

scores['Scores'] = 0

cols = ['Scores', 'Filename', 'Extracted Values']
scores = scores[cols]

In [ ]:
# evaluate scores for each file and generates evaluated.csv file ['Scores', 'Filename', 'Extracted Values']

# set filepath for a temp submission file.
filepath = r'F:\ML\HCL\HCL ML Challenge\auto_sub.csv'

idx = 0
f = 1

while(f == 1):
    for i, row in results.iloc[idx:].iterrows():
        print(i)
        f = 0
        df = pd.DataFrame(columns=['Filename', 'Extracted Values'])
        df['Filename'] = results.Filename

        df.iloc[i, 1] = row['Extracted Values']

        df.to_csv(filepath, index=False)

        upload = browser.find_element_by_xpath('//*[@id="upload-file-button-prediction-file-9c71a5552c0d4cc1b462711207a061ff"]')

        upload.click()
        time.sleep(1)
        
        try:
            autoit.win_active("Open")
            autoit.control_set_text("Open", "Edit1", filepath)
            autoit.control_send("Open", "Edit1", "{ENTER}")
        except:
            idx = i
            f = 1
            print("AutoItError at i = " + str(i))
            break
            
        time.sleep(7)

        score_board = browser.find_element_by_id('submission-result-poll-url')

        submission_id = score_board.get_attribute('url').split('/')[-3]

        print(submission_id)

        xp = r'//*[@id="submission-result-' + submission_id + r'"]/div/div[3]/div[2]'
    #     wait.until(EC.visibility_of_element_located((By.XPATH, xp)))
        try:
            score = browser.find_element_by_xpath('//*[@id="submission-result-' + submission_id + '"]/div/div[3]/div[2]')
            score = score.text
        except NoSuchElementException:
            idx = i
            f = 1
            print("NoSuchElementException at i = " + str(i))
            break
            
        scores.iloc[i, 0] = score

scores.to_csv('evaluated.csv', index=False)